# Softmax
-------------------------
## notMNIST dataset 
This is a classic case where a softmax regression is a natural, simple model. If you want to assign probabilities to an object being one of several different things, softmax is the thing to do, in addition when we train more sophisticated models, the final step will be a layer of softmax.

## Cross-entropy
We will be using the loss of our model with the following function

$$ H_{y'}(y) = -\sum_i y'_i \log(y_i) $$

Where is our predicted probability distribution, and is the true distribution (the one-hot vector with the digit labels).


### Visualize the dataset...
----------------------------

In [1]:
import numpy as np
import os

In [9]:
path = "./Datasets/notMNIST_small/{}/{}"

A = np.array(os.listdir("./Datasets/notMNIST_small/A/"))
B = np.array(os.listdir("./Datasets/notMNIST_small/B/"))
C = np.array(os.listdir("./Datasets/notMNIST_small/C/"))
D = np.array(os.listdir("./Datasets/notMNIST_small/D/"))
E = np.array(os.listdir("./Datasets/notMNIST_small/E/"))
F = np.array(os.listdir("./Datasets/notMNIST_small/F/"))
G = np.array(os.listdir("./Datasets/notMNIST_small/G/"))
H = np.array(os.listdir("./Datasets/notMNIST_small/H/"))
I = np.array(os.listdir("./Datasets/notMNIST_small/I/"))
J = np.array(os.listdir("./Datasets/notMNIST_small/J/"))

files = np.stack((A, B, C, D, E, F, G, H, I, J), axis=0)
labels = ["A", "B", "C", "D", "E","F", "G", "H", "I", "J"]

## The above ar np arrays holding the names of the files that contains each of the letters in notMNIST
print (path.format("A", A[5]))

./Datasets/notMNIST_small/A/Q2hvY0lDRy5vdGY=.png


In [3]:
## Take a look at the images
from PIL import Image
im = Image.open(path.format("A", A[5]))
im

In [4]:
def getPixels(im):
    pixels = list(im.getdata())
    width, height = im.size
    return np.array([pixels[i * width:(i + 1) * width] for i in range(height)]).reshape(width * height)

getPixels(im).shape

(784,)

In [5]:
def retrieveImages (array, letter):
    return np.array([getPixels(Image.open(path.format(letter, fname))) for fname in array])

retrieveImages(A, "A")

array([[  0,   0,   2, ..., 252, 255, 181],
       [  0,   1,   1, ..., 255, 255, 147],
       [  0,   0,   0, ..., 255, 255, 255],
       ..., 
       [  0,   0,   0, ..., 246, 252, 206],
       [  0,   0,   0, ...,   0,   3,   1],
       [  0,   0,   0, ..., 253, 255, 108]])

In [10]:
# Finally, all is ready to make our trainig matrix!
data = np.stack((retrieveImages(files[i], labels[i]) for i in range(10)), axis=0)

In [11]:
data.shape

(10, 1873, 784)